In [1]:
import numpy as np
from math import factorial
import matplotlib.pyplot as plt
from matplotlib import animation

## Eigenstates of the Harmonic oscillator

$$
|\psi_n(x)\rangle = \frac{1}{\pi 2^nn!} e^{-\frac{(x-x_0)^2}{2}} H_n\left(x-x_0 \right)
$$

where $H_n\left(x \right)$ are the Hermite polynomials of order $n$

NOTE: These are the eigenstates of the following Hamiltonian

$$
H=\frac{p^{2}}{2}+ \frac{1}{2}(x-x_{0})^{2}
$$

Any translation of $|\psi_n(x)\rangle \longrightarrow |\psi_n(x+\delta x)\rangle$ results in a state which is NO MORE an eigenstate of the original Hamiltonian.
We could then decompose $|\psi_n(x+\delta x)\rangle = \sum_{m}C_{m}|\psi_n(x)\rangle$ from some coefficients $C_{m}$ associated to the old eigenstates.

In [ ]:
import numpy as np
import math
import debugger as db
from scipy.special import factorial

def hermite(x, n):
    """
    hermite:
        Computes the Hermite polynomial of order 'n' over the real space grid 'x'.

    Parameters
    ----------
    x : np.ndarray
        Real space grid as a 1D numpy array.
    n : int
        Order of the Hermite polynomial. Must be a non-negative integer (n >= 0).

    Returns
    -------
    herm_pol : np.ndarray
        Hermite polynomial of order 'n', evaluated over the input grid 'x'.

    Pre-condition
    -------------
    - n >= 0. If n < 0, the function triggers a debugging checkpoint and halts execution.
    """
    # Pre-condition: n >= 0
    if n < 0:
        db.checkpoint(debug=True, msg=f"The order of the Hermite polynomial is not valid (n={n}, expected n>=0)", stop=True)

    # Coefficients set to 0 except for the one of order n.
    herm_coeffs = np.zeros(n + 1)
    herm_coeffs[n] = 1

    # Actual computation of the polynomial over the space grid.
    herm_pol = np.polynomial.hermite.hermval(x, herm_coeffs)
    return herm_pol


def stationary_state(x, omega, n):
    """
    harmonic_wfc:
        Computes the wavefunction of order 'n' for a quantum harmonic oscillator, 
        defined over the real space grid 'x'.

    Parameters
    ----------
    x : np.ndarray
        Real space grid as a 1D numpy array.
    omega : float
        Angular frequency of the harmonic oscillator. Must be positive (ω > 0).
    n : int
        Quantum number (wavefunction order). Must be n >= 0.

    Returns
    -------
    psi : np.ndarray
        Normalized wavefunction of order 'n', evaluated over the input grid 'x'.

    Pre-condition
    -------------
    - n >= 0. If n < 0, the function triggers a debugging checkpoint and halts execution.
    """
    # Constants set to 1 in atomic units.
    hbar = 1.0
    m = 1.0

    # Components of the analytical solution for stationary states.
    prefactor = 1 / np.sqrt(2**n * factorial(n, exact=False)) * ((m * omega) / (np.pi * hbar))**0.25
    x_coeff = np.sqrt(m * omega / hbar)
    exponential = np.exp(-(m * omega * x**2) / (2 * hbar))

    # Complete wavefunction.
    psi = prefactor * exponential * hermite(x_coeff * x, n)
    return psi

## Initialization for the simulation

Initialize correctly the real space and the momentum grid.
Be careful about the initialization of the momentum, how it should be treated.
Have a look at the `np.fft.fft` documentation.

<details>
  <summary>Solution part 1</summary>

```python
self.x = np.arange(-xmax + xmax / num_x, xmax, self.dx)
...
self.k = np.concatenate((np.arange(0, num_x / 2),
                                 np.arange(-num_x / 2, 0))) * self.dk
```
</details>

In [ ]:
class Param:
    """
    Container for holding all simulation parameters

    Parameters
    ----------
    xmax : float
        The real space is between [-xmax, xmax]
    num_x : int
        Number of intervals in [-xmax, xmax]
    dt : float
        Time discretization
    timesteps : int
        Number of timestep
    im_time : bool, optional
        If True, use imaginary time evolution.
        Default to False.
    """
    def __init__(self,
                 xmax: float,
                 num_x: int,
                 dt: float,
                 timesteps: int,
                 im_time: bool = False) -> None:

        self.xmax = xmax
        self.num_x = num_x
        self.dt = dt
        self.timesteps = timesteps
        self.im_time = im_time

        self.dx = 2 * xmax / num_x      # it's like 2L/N
        # Real time grid
        self.x = np.arange(-xmax + xmax / num_x, xmax, self.dx)
        # Definition of the momentum
        self.dk = np.pi / xmax          # k = pi / L
        # Momentum grid
        self.k = np.concatenate((np.arange(0, num_x / 2), np.arange(-num_x / 2, 0))) * self.dk


class Operators:
    """Container for holding operators and wavefunction coefficients."""
    def __init__(self, res: int) -> None:

        self.V = np.empty(res, dtype=complex)
        self.R = np.empty(res, dtype=complex)
        self.K = np.empty(res, dtype=complex)
        self.wfc = np.empty(res, dtype=complex)